# Use Case Description

Security teams often write detection rules or response playbooks (e.g., for SIEMs, SOAR platforms, or XDRs) but struggle to systematically map them to the right MITRE ATT&CK techniques. This limits the organization’s visibility into detection coverage and hinders frameworks like D3FEND, ATT&CK Navigator, and threat-informed defense.

The goal is to automatically analyze each detection play’s logic, objective, and context — and match it to the most relevant ATT&CK technique(s) with high confidence.

## Model used for this use case
Both Instruct Model and Reasoning Model can be used, but Instruct Model could work better as no complex reasoning is involved.

## SetUp

The setup scripts below are essentially the same as those in the [Quickstart (Instruct Model)](https://github.com/RobustIntelligence/foundation-ai-cookbook/blob/main/1_quickstarts/Preview_Quickstart_instruct_model.ipynb)

In [1]:
import os
import transformers
import torch
from IPython.display import display, Markdown

HF_TOKEN = os.getenv("HF_TOKEN")

def _get_device():
    if torch.cuda.is_available():
        return "cuda"
    elif torch.backends.mps.is_available():
        return "mps"
    else:
        return "cpu"

DEVICE = _get_device()

In [2]:
MODEL_ID = "fdtn-ai/Foundation-Sec-8B-Instruct"

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16, # this model's tensor_type is BF16
    token=HF_TOKEN,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
generation_args = {
    "max_new_tokens": 512,
    "temperature": None,
    "repetition_penalty": 1.2,
    "do_sample": False,
    "use_cache": True,
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.pad_token_id,
}

In [4]:
import re

def inference(prompt, system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    inputs = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    
    inputs = tokenizer(inputs, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_args,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens = False)
    
    # extract the assistant response part only
    match = re.search(r"<\|assistant\|>(.*?)<\|end_of_text\|>", response, re.DOTALL)
    
    return match.group(1).strip()

## MITRE Mapping Detection Plays

In [5]:
SYSTEM_PROMPT = "You are a cybersecurity analyst provided with a series of threat detection plays. Each play includes its name and a short objective."

def make_prompt(detection_plays):
    return f'''Your task: For each detection play, map it to the most relevant MITRE ATT&CK technique. Include:
    1. MITRE technique ID and name
    2. Justification for the mapping
    
    ## DETECTION PLAYS
    {detection_plays}
    
    Respond in a structured format:
    - Play Name: <name>
    - MITRE Technique: <TXXXX: Technique Name>
    - Reason: <why it's a fit>'''


In [6]:
detection_plays = """
- Name: Monitor Multiple Failed Login Attempts
  Objective: Detect brute-force attacks on user accounts by tracking repeated login failures.

- Name: Alert on PowerShell Script Execution
  Objective: Detect execution of potentially malicious PowerShell scripts by adversaries.

- Name: Dumping LSASS credentials
  Objective: Alert when attempts are made to dump credentials from LSASS process memory
"""


In [7]:
response = inference(make_prompt(detection_plays), SYSTEM_PROMPT)
display(Markdown(response))

Here is how you can structure your response based on the given information:

### Detection Plays Mapping

#### 1. **Play Name:** Monitor Multiple Failed Login Attempts  
   **MITRE Technique:** T1110.001: Password Guessing  
   **Reason:** This technique involves an adversary attempting various combinations of account names and passwords to gain access to valid accounts. Monitoring multiple failed logon attempts helps identify such activities that could indicate a brute force attempt to gain unauthorized access.

#### 2. **Play Name:** Alert on PowerShell Script Execution  
   **MITRE Technique:** T1059.003: Command and Scripting Interpreter: Windows Command Shell (PowerShell)  
   **Reason:** Adversaries use PowerShell as a command and scripting interpreter to execute commands, scripts, or loading modules. Alerts triggered upon script execution help detect this behavior which might be used for post-compromise behaviors like discovery, collection, or other techniques.

#### 3. **Play Name:** Dumping LSASS credentials  
   **MITRE Technique:** T1003.001: OS Credential Dumping: Security Account Manager  
   **Reason:** The goal here is to alert on actions that may lead to credential dumping from lsass.exe, specifically targeting the Security Accounts Manager (SAM). This aligns with the MITRE ATT&CK technique T1003.001 because it describes methods attackers use to obtain sensitive credential material by accessing the SAM database within the Local Security Authority Subsystem Service (LSASS).

Each entry maps a specific detection play to a corresponding MITRE ATT&CK technique, providing clarity on why these detections are effective against certain adversarial tactics, techniques, and procedures (TTPs).